In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time, os,requests
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

video_url = 'https://www.instagram.com/emmafituk_/?hl=en'
#create results folder if does not exists
res_folder=os.getcwd()+'/results'
os.makedirs(res_folder, exist_ok=True)
#initiate webdriver
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Optional: run in headless mode
prefs = {"download.default_directory" : res_folder}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("--headless=new")  # Use the newer headless mode
chrome_options.add_argument("--window-size=1920,1080")  # Set a large enough window size
chrome_options.add_argument("--disable-gpu")  # Sometimes useful in headless
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")


driver = webdriver.Chrome(options=chrome_options)
driver.get('https://fastdl.app/en')
driver.implicitly_wait(10) 

input_element=driver.find_element(By.CSS_SELECTOR,'input')
driver.implicitly_wait(10)
input_element.click()

driver.implicitly_wait(10) 
input_element.send_keys(video_url)
driver.implicitly_wait(10) 
button = driver.find_element(By.CSS_SELECTOR, 'button.search-form__button[type="submit"]')
button.click()
driver.implicitly_wait(10)
time.sleep(5)

body = driver.find_element("tag name", "body")
print(driver.execute_script("return document.body.scrollHeight"))

# Function to scroll down slowly
def smooth_scroll_down(driver, scroll_pause_time=0.2, pixels_per_scroll=100):
  current_scroll = 0  # Keep track of the current scroll position
  while current_scroll < 10000:
    driver.execute_script(f"window.scrollBy(0, {pixels_per_scroll});")
    current_scroll += pixels_per_scroll
    sleep(scroll_pause_time)

# Call the smooth scrolling function
smooth_scroll_down(driver)

print(driver.execute_script("return document.body.scrollHeight"))

time.sleep(5)
buttons = driver.find_elements(By.CSS_SELECTOR, "a.button.button--filled.button__download")
len(buttons)

In [ ]:

driver.quit()


In [ ]:
buttons[0].get_attribute('href')

In [ ]:
#import img in base 64
import base64
image_path = 'results/frames_DC7PCgOoF2t_2/DC7PCgOoF2t_0.jpg'
with open(image_path, "rb") as image_file:
    base64_string = base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
base64_string

In [ ]:

# Get list of all .jpg files in the directory
import os
directory = 'results/frames_DC7PCgOoF2t_2'
jpg_files = [file for file in os.listdir(directory) if file.endswith(".jpg")]

print("JPG files:", jpg_files)

In [ ]:
from utils.download_content import download_content
from utils.coach_ig_info import coach_ig_info
from utils.mp4_to_jpg import mp4_to_jpg
from utils.img_interpreter import img_interpreter
import os, shutil,base64


#specify the video url
coach_url = 'https://www.instagram.com/emmafituk_/?hl=en'
posts_info = coach_ig_info(coach_url.split('/')[-2])

In [ ]:
posts_info._date

In [ ]:
import subprocess, os, shutil
from moviepy.editor import VideoFileClip
#compute video duration
video_name='DC4_BEyRwbA_0.mp4'
video_filename = 'results/'+video_name
video_filename

In [ ]:
def mp4_to_jpg(video_name = 'DDDEwC4pVWm_0.mp4'):
  from moviepy.editor import VideoFileClip
  import subprocess, os, shutil

  #clean old results
  directory="results/frames_"+video_name[:-6]
  if os.path.exists(directory): shutil.rmtree(directory)
  os.mkdir(directory)

  video_filename = 'results/'+video_name

  #compute video duration
  video = VideoFileClip(video_filename)
  duration = video.duration
  #compute the frames seconds based on the video length
  seconds = [f"00:00:{x:02}" for x in [round(1 + i * (duration - 1) / 5) for i in range(6)]]

  for x,second in enumerate(seconds):
      command = 'ffmpeg -y -ss '+str(second)+' -i  '+video_filename +' -frames:v 1 '+directory+'/frame_'+str(x)+'.jpg'
      subprocess.run(command, shell=True, executable="/bin/bash",stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
      
  return directory

In [ ]:
mp4_to_jpg(video_name='DC4_BEyRwbA_0.mp4')

In [ ]:
os.remove('results/DC4_BEyRwbA_0.mp4')

In [ ]:
directory = 'results/frames_DC4_BEyRwbA'
images=[file for file in os.listdir(directory) if file.endswith(".jpg")]
images

In [ ]:
images_description = 'Summaries of the descriptions of the images in DC4_BEyRwbA content:\n\n'
for i,ip in enumerate(images):
    image_path=directory+'/'+ip
    with open(image_path, "rb") as image_file: base64_string = base64.b64encode(image_file.read()).decode('utf-8')
    img_description = img_interpreter(base64_string)
    images_description = images_description + 'Image_'+str(i)+ ':' +img_description+'\n\n'
    #delete the image
    os.remove(image_path)

In [ ]:
def download_content(video_url,coach_handel):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    import time, os,requests
    from bs4 import BeautifulSoup


    #create results folder if does not exists
    content_name = video_url.split('/')[-2]
    res_folder=os.getcwd()+'/results/'+str(coach_handel)+'/'+str(content_name)
    if os.path.exists(res_folder) and os.path.isdir(res_folder): shutil.rmtree(res_folder)
    os.makedirs(res_folder, exist_ok=True)
    #initiate webdriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Optional: run in headless mode
    prefs = {"download.default_directory" : res_folder}
    chrome_options.add_experimental_option("prefs",prefs)

    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://fastdl.app/en')
    driver.implicitly_wait(2) 
    
    while True:
        try:
            input_element=driver.find_element(By.CSS_SELECTOR,'input')
            driver.implicitly_wait(3)
            input_element.click()

            driver.implicitly_wait(3) 
            input_element.send_keys(video_url)
            driver.implicitly_wait(3) 
            button = driver.find_element(By.CSS_SELECTOR, 'button.search-form__button[type="submit"]')
            button.click()
            driver.implicitly_wait(3)
            time.sleep(2)
            break
        except:
            time.sleep(0.5)
            pass

    while True:
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR, "a.button.button--filled.button__download")
            button_list=[]
            for button in buttons:
                driver.implicitly_wait(5) 
                button_list.append(button.get_attribute('href'))
            for i,url in enumerate(button_list):
                if '.mp4' in url: extension='.mp4'
                else: extension='.jpg'
                r = requests.get(url)
                results_directory=res_folder+'/'+str(content_name)+'_'+str(i)+extension
                with open(results_directory, 'wb') as f:
                    f.write(r.content)
            Last_created_file=str(content_name)+'_'+str(i)+extension
            if Last_created_file in [f for f in os.listdir(res_folder) if os.path.isfile(os.path.join(res_folder, f))]:
                break
        except:
            time.sleep(0.5)
            pass


    driver.quit()
    files = [res_folder+'/'+f for f in os.listdir(res_folder) if os.path.isfile(os.path.join(res_folder, f))]
    return files

file_names=download_content(video_url='https://www.instagram.com/p/DDAVJ0DoyVc/?hl=en',coach_handel='emmafituk_')

In [ ]:
if '.mp4' in file_names[0]:
    print('ok!')

In [ ]:
file_names[0]


In [ ]:
directory=''
for i,f in enumerate(file_names[0].split('/')):
    if i != len(file_names[0].split('/'))-1:
        directory=directory+f+'/'
directory[:-1]

In [ ]:
from utils.coach_ig_info import coach_handles_all

coach_handles_all_l=coach_handles_all()
coach_urls = ['https://www.instagram.com/'+str(h)+'/?hl=en' for h in coach_handles_all_l.handle]

In [ ]:
def assign_category(frames_description):
    import openai
    from openai import OpenAI
    from dotenv import load_dotenv
    import os
    load_dotenv('.env')
    #post categories
    type_description = ''' 
    1. Entertainment: fits if the post seem to be mostly for entertainment purpose and is not directly selling anything nor giving useful tips or the like.
    2. About online coaching: fits if it describes what online coaching is, but is not outright trying to sell it.
    3. Promotion: is for a promotional content where the goal clearly is to try to sell or advertise a product (most likely online coaching or another type of programme).
    4. Educational/Tips: is for content that strives to teach about something be it nutrition, workouts, giving a recipe or other things, without focusing on selling something at the same time.
    5. Testimonials/Social Proof: is for a post that strives to show the effectiveness of something by showing or talking about an example of a person doing that thing - eg. transformation pictures, before and after weight or measurements or quotes from others about how well the thing worked for them.
    6. Mindset/Inspiration: is if the post focuses on one\’s mindset or strives to inspire.
    7. Brand/Personal is for content that is more specific to the sender whether it’s more personal content (eg. about the person writing the post, their friends or family, big events in their life) or very brand specific (eg. presenting an image for the brand, talking about their brand specifically).
    8. Other: is for a post that matches none of the categories listed.
    '''
    #Step5: analyze the video and assign a category
    prompt_video = "Given this definition of content types: " + type_description + '''Please evaluate this Social media content summary and fit it in only one type. 
    it is important that you only answer in the following dictionary format. Do not add anything to your answer: 
    {
    "Date of evaluation": "[Additional info date]",
    "Post date": "[Content Posted date]",
    "Coach name": "[name of the coach]",
    "Content id": "[content identifier]",
    "Post Type": "[Post Type name]",
    "Accuracy Post Type": "[percentage of accuracy]",
    "Comments count": "[number of comments]",
    "Likes": "[number of Likes]",
    "Views": "[number of views]",
    "Reactions": "[number of reactions]",
    "Shares": "[number of shares]",
    "Content short description": "[short summary of the content and explanation of your decision]"
    }
    '''

    api_key = os.getenv('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
    message='Content description: '+frames_description
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_video},  # Keep the text prompt as it is
                    {"type": "text", "text": message}  # Include any additional text instead of an image
                ]
            }
        ],
        temperature=0,
        max_tokens=200
    )
    input_tokens = response.usage.prompt_tokens  # Tokens used for the prompt
    output_tokens = response.usage.completion_tokens  # Tokens used for the completion

    # Pricing details for GPT-4o-mini
    input_cost_per_million = 0.15  # USD per million input tokens
    output_cost_per_million = 0.60  # USD per million output tokens
    # Pricing details for GPT-4-turbo
    input_cost_per_million = 10  # USD per million input tokens
    output_cost_per_million = 30  # USD per million output tokens


    # Calculate the cost for input and output tokens
    input_cost = (input_tokens / 1_000_000) * input_cost_per_million
    output_cost = (output_tokens / 1_000_000) * output_cost_per_million
    total_cost = input_cost + output_cost
    cost = f"Total estimated cost: ${total_cost:.4f}"
    return response.choices[0].message.content, cost

In [ ]:
with open('results/Full_info_on_runeraahauge.dk.txt', 'r', encoding='utf-8') as file: content = file.read()
assign_category(content)

In [ ]:
import pandas as pd, os, json
coaches_evaluation = txt_files = [f for f in os.listdir('results') if f.endswith('.txt')]
Coaches_content_type = pd.DataFrame()
for i, ce in enumerate(coaches_evaluation):
    with open('results/'+ce, 'r', encoding='utf-8') as file:
        content = file.read()
    Content_type = json.loads(assign_category(content))
    table_t=pd.DataFrame(Content_type,index=[i])
    Coaches_content_type = pd.concat([Coaches_content_type ,table_t])

Coaches_content_type.to_excel('Coaches_content_type.xlsx',index=False)

In [ ]:
Coaches_content_type.to_excel('Coaches_content_type.xlsx',index=False)

In [ ]:
Content_type['Post Type']
Content_type['Accuracy Post Type']

In [ ]:
import re
from collections import defaultdict

def parse_ctas(text):
    # Initialize dictionary to store the parsed data
    cta_dict = defaultdict(dict)

    # Regex patterns for parsing sections
    section_pattern = re.compile(r'^(.*? CTAs)', re.MULTILINE)
    cta_pattern = re.compile(r'(\d+)\.\s+"(.*?)"\n(.*?)(?=\n\d+\.|\n[A-Z].*? CTAs|$)', re.S)
    key_value_pattern = re.compile(r'(\w+):\s*(.*?)\n', re.S)
    aida_pattern = re.compile(r'AIDA:\s*(.*?)\nFormat:', re.S)

    # Split text by sections (Social Proof CTAs, Promotional CTAs, etc.)
    sections = section_pattern.split(text)
    sections = [s.strip() for s in sections if s.strip()]

    # Process each section
    for i in range(0, len(sections), 2):
        section_name = sections[i]
        ctas = sections[i + 1]

        # Find all CTAs within the section
        for match in cta_pattern.finditer(ctas):
            cta_number = int(match.group(1))
            title = match.group(2)
            rest = match.group(3).strip()

            # Extract Concept, Description, and other fields
            fields = {}
            for key_match in key_value_pattern.finditer(rest):
                key = key_match.group(1)
                value = key_match.group(2).strip()
                fields[key] = value

            # Extract AIDA fields
            aida_match = aida_pattern.search(rest)
            if aida_match:
                aida_text = aida_match.group(1)
                aida_fields = {}
                aida_lines = aida_text.split("\n")
                for line in aida_lines:
                    if ":" in line:
                        aida_key, aida_value = line.split(":", 1)
                        aida_fields[aida_key.strip()] = aida_value.strip()
                fields["AIDA"] = aida_fields

            # Add the CTA to the dictionary
            cta_dict[section_name][cta_number] = {
                "Title": title,
                **fields
            }

    return dict(cta_dict)

text= ''' 
Social Proof CTAs
1. "Best Transformations of 2024 Carousel"
Concept: Showcase top client transformations in an engaging carousel format.
Description: Share a carousel featuring your most impressive client transformations, with brief captions describing their journey and results.
AIDA:
Attention (Hook): "These are the BEST transformations of 2024—are YOU next?"
Interest: Highlight each client’s challenges and successes in a swipeable format.
Desire: Use before-and-after photos to inspire potential leads.
Action: "Click the link in my bio to hear more about how my online coaching can make 2025 your transformation year!"
Format: Carousel
CTA Method: Link in bio


2. "Storytelling Client Transformation"
Concept: Highlight a client’s transformation with their personal story.
Description: Share a detailed testimonial and transformation photos, focusing on the client’s journey from challenges to success.
AIDA:
Attention (Hook): "Meet [Client Name]—their story could be yours next!"
Interest: Share their journey, struggles, and how coaching helped them succeed.
Desire: Use relatable storytelling to inspire others to take the leap.
Action: "Click the link in my bio to start your own transformation journey!"
Format: Carousel
CTA Method: Link in bio

3. "Seasonal Success Stories"
Concept: Leverage client success stories tied to a seasonal theme.
Description: Highlight transformations that align with seasonal goals (e.g., people who signed up in January 2024 and how they look today).
AIDA:
Attention (Hook): "Where would you be in 1 year if today was your day 1?"
Interest: Show how your client achieved and maintained her transformation.
Desire: Share their journey and challenges to make it relatable.
Action: "Click the link in bio to hear more about my January transformation program!"
Format: Static Post or Carousel
CTA Method: Link in bio.


4. "Transformation Series"
Concept: Showcase client transformations to build credibility and inspire.
Description: Highlight before-and-after stories, focusing on their journey with your program.
AIDA:
Attention (Hook): "Imagine this being your transformation in 12 weeks!"
Interest: Explain the key steps and unique aspects of your program.
Desire: Use client testimonials about how their lives changed.
Action: "Click the link in my bio to start your transformation journey!"
Format: Carousel or Reel
CTA Method: Link in bio.

Promotional CTAs
5. "Limited-Time Offer" CTA
Concept: Create urgency with a time-limited opportunity.
Description: Encourage immediate sign-ups by offering a special program that’s only available for a short time.
AIDA:
Attention (Hook): "Only 3 days left to join the XX program"
Interest: Highlight what’s included in the program and the benefits.
Desire: Share testimonials or past results from clients who joined during similar promotions.
Action: "Click the link in bio to hear more about how my online coaching program can help you achieve your goals!!"
Format: Carousel or Reel
CTA Method: Link in bio.

6. 5 Months for the Price of 4 CTA
Concept: Reward commitment by offering a multi-month package with an extra month free.
Description: Encourage long-term consistency and progress by offering clients an additional month of coaching at no cost.
AIDA:
Attention (Hook): "Where could you be in 5 months if you started today?"
Interest: "To help you commit to your goals, I’m offering 5 months of coaching for the price of 4. You’ll get personalized plans, regular check-ins, and the consistency to see real, lasting results."
Desire: "Imagine stepping into summer feeling stronger, healthier, and more confident—all because you chose to commit to yourself now."
Action: "Comment ‘TODAY’ to hear more about how my online coaching program can help you achieve your goals!"
Format: Story or Reel
CTA Method: DM strategy (ManyChat or manual follow-up).

Lead Magnet CTAs
7. Free Recipe Guide
Concept: Offer a value-packed resource to attract leads.
Description: Share a free guide with easy, healthy recipes to help clients get started on their journey.
AIDA:
Attention: “Struggling with what to eat? Let me make it easier for you!”
Interest: “I’ve created a free guide packed with simple, delicious recipes to help you kick-start your health goals. These are meals my clients love because they’re easy to make and family-friendly!”
Desire: “Imagine staying on track without feeling restricted or overwhelmed. These recipes make healthy eating a breeze!”
Action: “Comment 2025 and I’ll send the guide via DM today!”
Format: Still Image or Carousel
CTA Method: DM strategy (ManyChat integration)

8. Free Meal Prep Checklist
Concept: Provide a tool to simplify meal planning and attract interest.
Description: Share a checklist that helps clients plan their meals for the week effortlessly.
AIDA:
Attention: “Meal prep doesn’t have to be stressful—grab this checklist to make it simple!”
Interest: “This free checklist will show you how to plan your meals for the week, save time, and stay on track with your goals.”
Desire: “With just 30 minutes of prep, you can save hours during the week and set yourself up for success.”
Action: “Comment ‘MEAL PREP’ or click the link in my bio to get your free checklist now!”
Format: Reel or Carousel
CTA Method: DM strategy (ManyChat or manual follow-up).

9. Coaching Giveaway
Concept: Create excitement with a chance to win free coaching or a related prize.
Description: Offer a giveaway for a free month of coaching to increase engagement and lead generation.
AIDA:
Attention: “Want to win a FREE month of coaching?”
Interest: “This giveaway includes everything you need: a personalized plan, weekly check-ins, and my full support to get you started on your journey.”
Desire: “Imagine kicking off your goals with expert guidance—at no cost!”
Action: “Comment ‘WIN’ below to enter, and I’ll DM you the details!”
Format: Still Image or Reel
CTA Method: DM strategy (ManyChat or manual follow-up))

10. Webinar CTA
Concept: Use an engaging, educational webinar to generate leads and provide value.
Description: Offer a live or recorded webinar teaching practical strategies while subtly promoting your coaching program.
AIDA:
Attention: “Want to learn the secrets to losing weight sustainably and achieving your new years resolution?”
Interest: *“Join my free 30-minute webinar where I’ll teach you:
The #1 mistake people make when starting their fitness journey in January
How to lose weight without giving up your favorite foods
A simple framework to stay consistent even on busy weeks.”
Desire: “You’ll leave with actionable tips to start seeing results right away—and a better understanding of how to reach your goals long term.”
Action: “Click the link in my bio to save your spot by XX date.”
Format: Reel or Carousel
CTA Method: Link in bio.

Humor & Creativity CTAs
11. “Sorry for the Side Effects” CTA
Concept: Use humor to highlight the positive changes clients experience through your program.
Description: Apologize playfully for the unexpected “problems” of success, like needing a new wardrobe or increased energy.
AIDA:
Attention: “Sorry in advance… you might need a new wardrobe soon! 😅”
Interest: “My clients keep telling me they’re ‘outgrowing’ their old clothes—literally! This program is designed to help you not just look better but feel amazing too.”
Desire: “Imagine feeling more confident and comfortable in your skin—this is the kind of problem you’ll want to have!”
Action: “Comment ‘WARDROBE’ to get started on your transformation today!”
Format: Reel or Carousel
CTA Method: DM strategy (ManyChat or manual follow-up).


12. “Smashing the Scale” CTA
Concept: Playfully challenge unhealthy obsessions with weight by encouraging focus on real progress.
Description: Use creative visuals or metaphors to make a bold statement about ditching the scale.
AIDA:
Attention: “Your scale doesn’t define you—it’s time to smash that mindset!”
Interest: “In my program, we focus on what really matters: how you feel, your energy, and sustainable habits. Numbers can’t tell the whole story of your progress.”
Desire: “Imagine loving the process and seeing results you can feel—not just measure. This is about becoming your best self.”
Action: “Click the link in my bio to hear more about how you can smash your goals this year!”
Format: Reel or Carousel
CTA Method: Link in bio.
13. “The Metaphor” CTA
Concept: Use a fun and creative metaphor to illustrate weight-loss success.
Description: Compare weight lost or strength gained to everyday objects for a playful, relatable hook.
AIDA:
Attention: “Ever heard of losing a washing machine? My client did!”
Interest: “In just a few months, they lost over 50 pounds—the equivalent of a washing machine! This program is about real, life-changing results.”
Desire: “Imagine celebrating your own transformation with stories that make you smile. What could your ‘washing machine’ be?”
Action: “Click the link in my bio to hear more about how you can achieve your 2025 goals through my online coaching program”
Format: Carousel or Reel
CTA Method: Link in bio.

14. Post-Holiday Humor CTA
Concept: Relate to common struggles after holidays with a playful, lighthearted tone.
Description: Use humor to connect with your audience while offering a solution to get back on track.
AIDA:
Attention: “Feel like you ate Santa’s entire cookie stash this holiday season?”
Interest: “Don’t worry—you’re not alone. Let’s hit reset together and start feeling good again.”
Desire: “With my program, you’ll find balance and progress without feeling deprived. The holiday fun doesn’t have to hold you back.”
Action: “Click the link in my bio, and save the setup fee for the next 24 hrs.”
Format: Reel or Carousel
CTA Method: Link in bio.


Seasonal and Goal-Oriented CTAs
15. “New Year, New You” CTA
Concept: Inspire action by leveraging the New Year as a fresh start.
Description: Encourage people to set meaningful goals and commit to your coaching program as part of their New Year’s resolutions.
AIDA:
Attention: “This is YOUR year. Are you ready to make it happen?”
Interest: “New Year’s resolutions don’t have to fail. With my personalized coaching program, you’ll have the tools, guidance, and accountability to see real progress—and love the process.”
Desire: “Imagine looking back in December knowing this was the year you made yourself a priority. It starts with a single decision today.”
Action: “Comment ‘NEW YEAR’ or click the link in my bio to secure your spot in my January program!”
Format: Carousel or Reel
CTA Method: DM strategy (ManyChat or manual follow-up).

16. “With vs. Without a Coach” CTA
Concept: Show the difference coaching can make by comparing potential outcomes.
Description: Use a relatable comparison to highlight how your coaching program supports consistent results. 
AIDA:
Attention: “What does your 2025 look like—WITH a coach or WITHOUT one?”
Interest: “Without a coach: Unclear goals, inconsistent progress, and a lot of frustration. With a coach: A clear plan, accountability, and confidence in your results.”
Desire: “Imagine where you could be in a few months with the right guidance and support. You don’t have to do it alone.”
Action: “Comment ‘COACH’ or click the link in my bio to see how I can help you reach your goals this year.”
Format: Carousel or Reel
CTA Method: DM strategy (ManyChat or manual follow-up).


Education/Authority CTAs
17. Interactive Post CTA
Concept: Engage your audience by inviting them to share their goals or opinions.
Description: Encourage followers to participate in the conversation, creating a sense of inclusion and connection. DM the first XX people who write a tip and get the conversation started in the DM about their 2025 goals.
AIDA:
Attention: “What’s your BIGGEST fitness goal for 2025?”
Interest: “I love hearing about your goals—it’s the first step to making them real. Whether it’s running your first 5K, feeling more confident, or simply building healthy habits, every goal matters!”
Desire: “Sharing your goal is the first step toward making it happen—and I’m here to support you along the way.”
Action: “Comment below and let me know what you’re working toward. I’ll DM you a free tip to help you get started!”
Format: Carousel or Story
CTA Method: Comment-based DM (ManyChat or manual follow-up).

18. “Seasonal Recipe Post” CTA
Concept: Share valuable content by posting a seasonal recipe that aligns with your coaching style.
Description: Provide a free recipe to engage your audience and build trust through helpful, actionable content. Add a CTA in the recipe DM. 
AIDA:
Attention: “Need a healthy but delicious dish for winter dinners? Try this!”
Interest: “This [recipe name] is perfect for staying on track during colder months—comfort food that’s easy, filling, and packed with nutrients.”
Desire: “Eating healthy doesn’t have to be complicated or boring. My clients love recipes like this because they’re simple and family-friendly.”
Action: “Comment ‘RECIPE’ and I’ll send it to you for free!”
Format: Reel or Carousel
CTA Method: DM strategy (ManyChat or manual follow-up).

19.Content Series CTA
Concept: Use a recurring series like “Workout Wednesday” or “Mythbusters” to educate and engage your audience.
Description: Share effective workouts or debunk common fitness and nutrition myths to position yourself as an expert.
AIDA:
Attention: “Is eating carbs at night really that bad for you? Let’s bust this myth.”
Interest: “Here’s the truth: Your body doesn’t know what time it is when it processes food. It’s more about overall calories and nutrient balance than what time you eat.”
Desire: “Knowing the facts can help you make choices without fear or restriction—this is the foundation of my coaching approach.”
Action: “Visit my website to learn how my coaching can help you achieve your goals with clarity and confidence.”
Format: Carousel or Reel
CTA Method: Link in Bio

Competition CTAs
20. “Transformation Challenge” CTA
Concept: Inspire action through a competitive challenge that rewards commitment and results.
Description: Offer a limited-time transformation challenge where participants can compete for prizes while achieving their goals.
AIDA:
Attention: “Are you ready to transform your life AND win amazing prizes?”
Interest: “Join my [Name of Challenge]—a 6-week program designed to help you achieve your fitness goals with personalized plans and weekly check-ins. Top performers will win [specific prizes]!”
Desire: “Imagine seeing real results while competing with others for something fun and motivating—it’s a win-win!”
Action: “Click the link in my bio to hear more about the new program!”
Format: Reel or Carousel
CTA Method: Link in bio.

21. “Bootcamp” CTA
Concept: Promote a 4-month, high-impact program focused on a specific goal (e.g., toning, weight loss).
Description: Encourage participants to commit to a focused challenge for measurable results in just 4 months.
AIDA:
Attention: “Want to tone up and feel amazing before spring 2025? Let’s do it together!”
Interest: “Join my Booty Camp, where you’ll receive targeted workouts, personalized meal plans, and daily accountability to keep you on track.”
Desire: “Picture how much stronger, more confident, and healthier you’ll feel by the end of these 4 months. It’s not just about the results—it’s about how you’ll feel showing up for yourself.”
Action: “Click the link in my bio to hear more about the booty camp!”
Format: Still Image or Reel
CTA Method: Link in bio.

22. “Win Free Coaching” CTA
Concept: Boost engagement with a giveaway for a chance to win a free month of coaching.
Description: Run a giveaway where one lucky participant wins a free month of your coaching program, encouraging interaction and new leads.
AIDA:
Attention: “Want to win FREE coaching for a month?”
Interest: “This giveaway includes everything you need: personalized plans, weekly check-ins, and my full support to get you closer to your goals.”
Desire: “Imagine having expert guidance completely free as you kick off your fitness journey—this could be your chance!”
Action: “Comment ‘WIN’ below to enter and stay tuned—I’ll announce the winner on [specific date]!”
Format: Reel or Still Image
CTA Method: Comment-based DM (ManyChat or manual follow-up).

Scarcity/Exclusivity CTAs
23. “Limited Spots Available” CTA
Concept: Leverage exclusivity and scarcity to drive urgency while evoking an emotional response tied to frustration, hope, and taking the first step toward real change.
Description: Highlight a limited number of spots with an emotional appeal to clients who are ready to stop feeling stuck and start achieving their goals.
AIDA:
Attention: “You’ve tried everything, but nothing seems to stick. You’re not alone—and it’s not your fault. But what if you could change that starting today?”
Interest: “There are just 7 spots left in my January coaching program. This isn’t a generic plan or a quick fix. It’s tailored to you - personalized meal plans, effective workouts, and weekly check-ins to keep you accountable every step of the way.”
Desire: “Imagine where you could be 3 months from now. Stronger, more confident, and finally free of the frustration of going it alone. This is your time to make a change.”
Action: “Click the link in my bio to claim your spot and start building the life you deserve. Don’t wait—these will fill fast!”
Format: Story or Carousel
CTA Method: Link in bio.

24. “Free Month of Coaching” CTA
Concept: Use a short-term, exclusive offer to reward commitment and attract sign-ups.
Description: Offer a free month of coaching for clients who commit to a long-term plan within a limited time.
AIDA:
Attention: “Want to start strong this year? Your first month is on me!”
Interest: “This week only, I’m offering a free month of coaching for anyone who signs up for a 4-month plan or longer. This is your chance to start with guidance and support without the initial cost.”
Desire: “Imagine starting today and seeing results, knowing you’ve saved and taken the first step to feeling your best.”
Action: “Comment ‘FREE MONTH’ or click the link in my bio to claim your spot before XX date!”
Format: Reel or Carousel
CTA Method: DM strategy (ManyChat or manual follow-up).

25. “I’m Looking For…” CTA
Concept: Flip the dynamic of traditional CTAs by positioning clients as applicants to your exclusive coaching program.
Description: Use a clear set of requirements to target a specific audience and position your coaching program as a highly desirable opportunity.
AIDA:
Attention: “Are you [specific target audience description] who would love to [achieve goal] or is struggling with [specific challenge/problem]? If so, I’ve got something for you!”
Interest: “My name is [Coach Name], and over the past [X] years, I’ve helped more than [X] individuals reach their goals, including [target audience’s biggest goal]. I’ve just opened up [X] spots for my new online coaching program, and I’m offering an early bird discount of 30% off—but to join, you’ll need to say YES to the following:”
Desire:
You are between [X and Y] years old.
You have a dream of [specific goal, e.g., losing 10-30 pounds].
You have a busy lifestyle and need a plan that works for YOU.
You’re willing to invest in yourself mentally, with your time, and financially (equivalent to $5-$10 per day).
“If this sounds like you, this is your moment to make a change!”
Action: “Click the link in my bio to fill out the lead form, and let’s discuss your goals. I can’t wait to see what we can achieve together!”
Format: Carousel post featuring bold text and client transformations.
CTA Method: Link in bio.

Objection Handling CTAs
26. “Reframe the Mindset” CTA
Concept: Address mindset challenges by reframing fitness and health as a positive, enjoyable journey.
Description: Tackle objections by showing how your program turns fitness into a sustainable, fulfilling lifestyle.
AIDA:
Attention: “Tired of workouts that feel like punishment? It doesn’t have to be this way!”
Interest: “My coaching program is built around making fitness enjoyable and sustainable—no crazy restrictions, no burnout. Just a plan that works for YOU.”
Desire: “Imagine loving the process and seeing progress that feels easy and natural. You’ll wake up excited to move and feel proud of the changes you’re making.”
Action: “Comment ‘JOY’ to learn how we can build a fitness plan that fits your life and goals!”
Format: Story or Reel
CTA Method: DM strategy (ManyChat or manual follow-up).

27.“Reframe Your Investment” CTA
Concept: Reframe the cost of coaching as an investment in one’s health and confidence.
Description: Show clients the long-term value of investing in coaching compared to short-term solutions that don’t last.
AIDA:
Attention: “Is coaching worth it? Let’s talk about that…”
Interest: “You might be thinking, ‘I can’t afford this,’ but let me ask you—what would it be worth to feel confident in your body, energized every day, and in control of your health?”
Desire: “Investing in your health now means saving time, money, and frustration later. My coaching isn’t just about weight loss—it’s about changing your life.”
Action: “Comment ‘INVEST’ to hear more about online coaching.”
Format: Carousel or Still Image
CTA Method: DM strategy (ManyChat or manual follow-up).

28. “No Time? No Problem” CTA
Concept: Address objections about time constraints by showing how your program fits into busy schedules.
Description: Highlight how personalized coaching can help clients make progress even with limited time.
AIDA:
Attention: “Too busy to work on your goals? I get it—let’s find a way that works.”
Interest: “My program is designed for people with packed schedules. You’ll get quick, effective workouts, flexible meal plans, and strategies to maximize your time.”
Desire: “Imagine seeing results without spending hours in the gym or overhauling your routine. This program fits into YOUR life, not the other way around.”
Action: “Comment ‘TIME’ to see how I can help you hit your goals, even with a busy schedule!”
Format: Reel or Carousel
CTA Method: DM strategy (ManyChat or manual follow-up).



29. It’s Cheaper Than You Think!
Concept: Reframe the cost of coaching in a relatable way, comparing it to an everyday habit like coffee.
Description: Use a coffee comparison to show how coaching is affordable and valuable for long-term health.
AIDA:
Attention: “Think coaching is expensive? Let me show you something…”
Interest: “Most people spend $6 a day on coffee. For about the same cost, you could have a coaching program that gives you energy, confidence, and results that last. A cup of coffee feels good for a few hours, but imagine how it feels to wake up energized every day, proud of the progress you’ve made.”
Desire: “My coaching program costs about the same as your daily coffee habit—but the results will last a lifetime. Clients tell me it’s the best investment they’ve ever made.”
Action: “Click the link in my bio to sign up before Sunday and save $100—you can have your coffee and your transformation!”
Format: Reel or Carousel
CTA Method: Link in bio.

Personal/BTS CTAs
30. Personal Story CTA
Concept: Use your own transformation or journey to inspire and connect with your audience.
Description: Share your personal struggles, challenges, and successes to build trust and relatability.
AIDA:
Attention: “I used to think I’d never feel confident in my own skin—here’s how I changed that.”
Interest: “Years ago, I was stuck in a cycle of yo-yo dieting and inconsistent workouts. I thought I was doing everything right, but nothing stuck. Then I found a system that worked for ME, and everything changed.”
Desire: “Now I feel stronger, healthier, and more confident than ever, and I’m here to help YOU feel the same way.”
Action: “Click the link in my bio to learn more about how I can help you transform your health and confidence.”
Format: Reel or Carousel
CTA Method: Link in bio.


31. Behind-the-Scenes CTA
Concept: Provide a peek into your coaching process to build transparency and connection.
Description: Show what it’s like to work with you by sharing snippets of your day or your clients’ journeys.
AIDA:
Attention: “Ever wonder what it’s like to have a coach by your side?”
Interest: “Here’s a sneak peek into my coaching process—how I create personalized plans, provide weekly accountability, and celebrate every win with my clients.”
Desire: “You deserve to feel supported, seen, and empowered to make progress. That’s what I’m here for.”
Action: “Click the link in my bio to see how I can help you feel supported and confident in your journey!”
Format: Story or Reel
CTA Method: Link in bio.

32. Coaching Philosophy CTA
Concept: Share your unique approach to coaching to resonate with your ideal clients.
Description: Explain your values and methods in a relatable way to demonstrate why your program works.
AIDA:
Attention: “Coaching isn’t about telling you what to do—it’s about helping you find what works for YOU.”
Interest: “My coaching philosophy is simple: this is your journey, and I’m here to guide you. Whether it’s flexible meal plans, time-efficient workouts, or mindset shifts, we’ll find what fits into your life—not the other way around.”
Desire: “You don’t need to feel restricted or overwhelmed to make progress. My clients feel empowered to build habits that stick—and you can too.”
Action: “Click the link in my bio to join my program and start building habits that work for your life.”
Format: Carousel or Still Image
CTA Method: Link in bio.

33. What Would Your Older Self Thank You For?
Concept: Encourage your audience to think long-term and take action today for a healthier future.
Description: Use the perspective of the future self to inspire immediate action and commitment.
AIDA:
Attention: “Imagine your 80-year-old self looking back—what would they thank you for?”
Interest: “Are you a busy [target audience] who’s been putting off your health and fitness goals? Here’s your reminder: your future self will thank you for taking control of your health today. Imagine looking back and realizing that prioritizing your body allowed you to live your life fully, with energy and confidence.”
Desire: “For the past [X] years, I’ve helped hundreds of [target audience] build sustainable habits that keep them strong and healthy for life. Health is wealth—and nothing else matters as much if you’re not feeling your best.”
Action: “Click the link in my bio to sign up within 24 hours, save the setup fee, and take the first step toward a healthier and fitter future self!”
Format: Reel or Still Image
CTA Method: Link in bio.
'''
cta_data = parse_ctas(text)


In [ ]:
from utils.aida_cta_definitions import aida_cta_definitions
aida_cta_definitions['Social Proof CTAs']

In [ ]:
from utils.aida_cta_definitions import parse_ctas

cta_data = parse_ctas()
cta_data

In [ ]:
def assign_aida_category(frames_description,model="gpt-4o-mini"):
    from openai import OpenAI
    from utils.check_cost import check_cost
    from dotenv import load_dotenv
    from utils.aida_cta_definitions import aida_cta_definitions
    import os
    load_dotenv('.env')
    #post categories
    type_description = str(aida_cta_definitions)
    #Step5: analyze the video and assign a category
    prompt_video = "Given this 33 definitions of CTAs: " + type_description + '''Please evaluate this Social media content summary and fit it in only one of the 33 types.
    In making your choice, please pay attention at the summaries of the descriptions of the images, Caption, Post Type and Post Format
    it is extremely important that you only answer in the following dictionary format. Do not add or subtract anything to your answer: 
    {
    "CTA name": "[the most appropriate cta among the 33 types (e.g. '32. Coaching Philosophy CTA')]",
    "CTA category accuracy": "[How accure you think your choice is (please express the value in %)]",
    "CTA category explanation": "[Please explain the reasoning for this choice]",
    "CTA improvement": "[please explain how the Content could improve to get closer to the perfect selected CTA as described before ]
    }

    '''

    api_key = os.getenv('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
    message='Content description: '+frames_description
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_video},  # Keep the text prompt as it is
                    {"type": "text", "text": message}  # Include any additional text instead of an image
                ]
            }
        ],
        temperature=0,
    )
    cost = check_cost(response, model)
    return response.choices[0].message.content, cost

with open("results/Full_info_on_fitbywh.txt", "r") as file:
    content = file.read()
import pprint
pprint.pprint(assign_aida_category(content))

In [ ]:
aida_cta_definitions
import pandas as pd
tab_t = pd.DataFrame(aida_cta_definitions['Social Proof CTAs']).T


In [15]:
from utils.aida_cta_definitions import aida_cta_definitions
import pandas as pd
tab = pd.DataFrame()
for k in aida_cta_definitions.keys():
    tab_t = pd.DataFrame(aida_cta_definitions[str(k)])
    tab = pd.concat([tab,tab_t])
tab = tab.reset_index(drop=True)

new_list = [str(cn).replace('"','').replace("'",'').replace('“','').replace('”','') for cn in tab['Title']]
tab.columns = ['Title_recommended', 'Concept_recommended', 'Description_recommended', 'AIDA_recommended', 'Format_recommended', 'CTA_Method_recommended']
tab['Title_recommended'] = new_list

Coaches_summaries = pd.read_excel('results/Coaches_overall_summary.xlsx')
new_list_1 = [str(cn).replace('"','').replace("'",'').replace('“','').replace('”','') for cn in Coaches_summaries['CTA name']]
Coaches_summaries['CTA name'] = new_list_1

Coaches_summaries = pd.merge(Coaches_summaries,tab, left_on='CTA name',right_on='Title_recommended', how='left')

,Date of evaluation,Post date,Coach name,Content id,Post Type,Accuracy Post Type,Comments count,Likes,Views,Reactions,...,CTA name,CTA category accuracy,CTA category explanation,CTA improvement,Title,Concept,Description,AIDA,Format,CTA Method
0,2024-12-10,2024-12-09,fitbywh,DDXz-o5t1av,Promotion,95%,94,30,1175.0,30,...,7. Free Recipe Guide,90%,The content primarily focuses on inviting view...,To enhance alignment with the Free Recipe Guid...,7. Free Recipe Guide,Offer a value-packed resource to attract leads.,"Share a free guide with easy, healthy recipes ...",{'Attention (Hook)': 'Struggling with what to ...,Still Image or Carousel,DM strategy (ManyChat integration)
1,2024-12-10,2024-12-09,larsencoaching,DDXGyqPNcLz,Promotion,95%,8,126,21343.0,126,...,26. Reframe the Mindset CTA,90%,The content summary reflects a humorous and re...,To improve the content and align it more close...,26. Reframe the Mindset CTA,Address mindset challenges by reframing fitnes...,Tackle objections by showing how your program ...,{'Attention (Hook)': 'Tired of workouts that f...,Story or Reel,DM strategy (ManyChat or manual follow-up)
2,2024-12-10,2024-12-09,actionanton,DDXFbZ-sveu,Educational/Tips,95%,2,27,NaN,27,...,18. Seasonal Recipe Post CTA,90%,The content focuses on promoting a healthier r...,To improve the content's alignment with the se...,18. Seasonal Recipe Post CTA,Share valuable content by posting a seasonal r...,Provide a free recipe to engage your audience ...,{'Attention (Hook)': 'Need a healthy but delic...,Reel or Carousel,DM strategy (ManyChat or manual follow-up)
3,2024-12-10,2024-12-09,borkfitness,DDXONEGoUXf,Educational/Tips,95%,0,84,8551.0,84,...,19. Content Series CTA,90%,The content features a Q&A format where the co...,To enhance the alignment with the selected CTA...,19. Content Series CTA,Use a recurring series like 'Workout Wednesday...,Share effective workouts or debunk common fitn...,{'Attention (Hook)': 'Is eating carbs at night...,Carousel or Reel,Link in Bio
4,2024-12-10,2024-12-09,mikkeldove,DDXRLwkoif6,Educational/Tips,100%,0,33,4759.0,33,...,19. Content Series CTA,90%,The content focuses on providing educational t...,To enhance the alignment with the selected CTA...,19. Content Series CTA,Use a recurring series like 'Workout Wednesday...,Share effective workouts or debunk common fitn...,{'Attention (Hook)': 'Is eating carbs at night...,Carousel or Reel,Link in Bio
5,2024-12-10,2024-12-09,runeraahauge.dk,DDXnBPwObJP,Educational/Tips,95%,0,35,3207.0,35,...,19. Content Series CTA,90%,The content focuses on an educational tip rega...,To enhance the alignment with the selected CTA...,19. Content Series CTA,Use a recurring series like 'Workout Wednesday...,Share effective workouts or debunk common fitn...,{'Attention (Hook)': 'Is eating carbs at night...,Carousel or Reel,Link in Bio
6,2024-12-10,2024-12-09,sofiaakumasci,DDXIfqxNZch,Testimonials/Social Proof,95%,2,54,0.0,54,...,12. Smashing the Scale CTA,90%,The content focuses on the transformation of a...,To improve the alignment with the selected CTA...,12. Smashing the Scale CTA,Playfully challenge unhealthy obsessions with ...,Use creative visuals or metaphors to make a bo...,{'Attention (Hook)': 'Your scale doesn’t defin...,Reel or Carousel,Link in bio
7,2024-12-10,2024-12-09,anderstrust,DDXSk8jtaI4,Mindset/Inspiration,85%,3,81,7893.0,81,...,26. Reframe the Mindset CTA,90%,The content focuses on addressing common misco...,"To enhance alignment with the selected CTA, th...",26. Reframe the Mindset CTA,Address mindset challenges by reframing fitnes...,Tackle objections by showing how your program ...,{'Attention (Hook)': 'Tired of workouts that f...,Story or Reel,DM strategy (ManyChat or manual follow-up)
8,2024-12-10,2024-11-29,abdelloo,DC8zLFFoYuj,Educational/Tips,95%,0,38,7767.0,38,...,19. Content Series CTA,90%,The content focuses on debunking common miscon...,To enhance the alignment with the selected CTA...,19. Content Series CTA,Use a recurring series like 'W

Index(['Title', 'Concept', 'Description', 'AIDA', 'Format', 'CTA Method'], dtype='object')